I use FoCS environment. Python 3.11.6 version.

In [43]:
# Import useful libraries
import os
import json
import re
import pandas as pd
import numpy as np
from collections import defaultdict

In [23]:
# Configuration file
with open('config.json', 'r') as f:
    config = json.load(f)

You have to work on the [ZTBus: A Large Dataset of Time-Resolved City Bus Driving Missions](https://www.research-collection.ethz.ch/handle/20.500.11850/626723) repository.

It contains:
*  [metaData.csv](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv?sequence=1&isAllowed=y), shortly *trips*
*  several other files containing detailed data on some bus parameters, whose name is in the *trips* file. Those files can be downloaded as a [zip file](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_compressed.zip?sequence=3&isAllowed=y). Let us call those datasets the *details* datasets.

# Import

**ZT_bus Folder Structure**: The 'ZT_bus' folder is the main directory containing data related to driving missions.
Inside this folder, you'll find information on 1409 driving missions.

**Metadata Information**: Additionally, there is a 'metadata' folder that stores metadata related to these missions.
The metadata includes information for all 1490 missions.


## Metadata csv

In [44]:
metadata = pd.read_csv(config['path_metadata'])

In [45]:
metadata.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [5]:
metadata.shape

(1409, 16)

In [26]:
# Check metadata dtypes
metadata.dtypes

name                             object
busNumber                         int64
startTime_iso                    object
startTime_unix                    int64
endTime_iso                      object
endTime_unix                      int64
drivenDistance                  float64
busRoute                         object
energyConsumption               float64
itcs_numberOfPassengers_mean    float64
itcs_numberOfPassengers_min       int64
itcs_numberOfPassengers_max       int64
status_gridIsAvailable_mean     float64
temperature_ambient_mean        float64
temperature_ambient_min         float64
temperature_ambient_max         float64
dtype: object

`startTime_iso` has `object` format. It should be a `datetime` object. Later, when this variable will be used for analysis it will be better to change the format or to use `startTime_unix`. The same regards the `endTime_iso` variable. The other format are correct for the nature of the variable they represent. 

TODO: Check if other data types should be different.

### Nan values check

In [29]:
metadata.isna().sum()

name                            0
busNumber                       0
startTime_iso                   0
startTime_unix                  0
endTime_iso                     0
endTime_unix                    0
drivenDistance                  0
busRoute                        0
energyConsumption               0
itcs_numberOfPassengers_mean    0
itcs_numberOfPassengers_min     0
itcs_numberOfPassengers_max     0
status_gridIsAvailable_mean     0
temperature_ambient_mean        0
temperature_ambient_min         0
temperature_ambient_max         0
dtype: int64

In [31]:
print(f'Metadata dataframes has: {metadata.isna().sum().sum()} null values.')

Metadata dataframes has: 0 null values.


Checking the output of before, the variable `busRoute` should have nan values because from `metadata.head()` I see that some cells have `-` corresponding to a missing value. If this variable will be used for analysis I have to take in consideration this aspect. 

## Import ZT_bus data

In [27]:
file_names = [file_name for file_name in os.listdir(config['path_ZTbus_folder']) if 
              os.path.isfile(os.path.join(config['path_ZTbus_folder'], file_name))]

In [109]:
# Print only the first 10 results
file_names[:10]

['B183_2020-11-13_14-52-45_2020-11-13_19-13-45.csv',
 'B183_2020-10-06_04-23-44_2020-10-06_07-33-54.csv',
 'B183_2019-10-03_03-04-42_2019-10-03_18-38-45.csv',
 'B183_2022-05-02_03-02-19_2022-05-02_17-07-49.csv',
 'B183_2021-04-23_03-47-54_2021-04-23_07-48-48.csv',
 'B208_2022-08-15_03-31-51_2022-08-15_12-34-10.csv',
 'B183_2020-07-24_04-01-31_2020-07-24_18-04-39.csv',
 'B183_2022-10-28_13-36-23_2022-10-28_16-37-08.csv',
 'B208_2021-04-21_04-10-07_2021-04-21_18-19-32.csv',
 'B183_2022-07-28_14-27-33_2022-07-28_19-17-23.csv']

In [10]:
len(file_names) == metadata.shape[0]

True

As expected, ZT_bus folder contains data related to 1409 driving missions and metadata contains information related to that missions (always 1490).

In [28]:
dataframes = {}
for el in file_names:
    dataframes[el[:-4]] = pd.read_csv(f'{config["path_ZTbus_folder"]}/{el}')

In [12]:
type(dataframes)

dict

`dataframes` is a dictionary of dataframes, in total 1490. Each of them correspond to a driving mission.

In [111]:
# Print the first 10 keys
list(dataframes.keys())[:10]

['B183_2020-11-13_14-52-45_2020-11-13_19-13-45',
 'B183_2020-10-06_04-23-44_2020-10-06_07-33-54',
 'B183_2019-10-03_03-04-42_2019-10-03_18-38-45',
 'B183_2022-05-02_03-02-19_2022-05-02_17-07-49',
 'B183_2021-04-23_03-47-54_2021-04-23_07-48-48',
 'B208_2022-08-15_03-31-51_2022-08-15_12-34-10',
 'B183_2020-07-24_04-01-31_2020-07-24_18-04-39',
 'B183_2022-10-28_13-36-23_2022-10-28_16-37-08',
 'B208_2021-04-21_04-10-07_2021-04-21_18-19-32',
 'B183_2022-07-28_14-27-33_2022-07-28_19-17-23']

In [15]:
len(dataframes.keys())

1409

Let's take two samples dataframes to analyse.

In [32]:
dataframes['B183_2019-04-30_03-18-56_2019-04-30_08-44-20'].head()

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30T03:18:58Z,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0
3,2019-04-30T03:18:59Z,1556594339,-1.087931,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,255000.0,0.0
4,2019-04-30T03:19:00Z,1556594340,-0.811985,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,253000.0,0.0


In [18]:
dataframes['B183_2019-04-30_03-18-56_2019-04-30_08-44-20'].columns

Index(['time_iso', 'time_unix', 'electric_powerDemand', 'gnss_altitude',
       'gnss_course', 'gnss_latitude', 'gnss_longitude', 'itcs_busRoute',
       'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce'],
      dtype='object')

In [19]:
dataframes['B183_2019-04-30_03-18-56_2019-04-30_08-44-20'].shape

(19525, 26)

In [23]:
dataframes['B183_2019-04-30_03-18-56_2019-04-30_08-44-20'].dtypes

time_iso                       object
time_unix                       int64
electric_powerDemand          float64
gnss_altitude                 float64
gnss_course                   float64
gnss_latitude                 float64
gnss_longitude                float64
itcs_busRoute                  object
itcs_numberOfPassengers       float64
itcs_stopName                  object
odometry_articulationAngle    float64
odometry_steeringAngle        float64
odometry_vehicleSpeed         float64
odometry_wheelSpeed_fl        float64
odometry_wheelSpeed_fr        float64
odometry_wheelSpeed_ml        float64
odometry_wheelSpeed_mr        float64
odometry_wheelSpeed_rl        float64
odometry_wheelSpeed_rr        float64
status_doorIsOpen               int64
status_gridIsAvailable          int64
status_haltBrakeIsActive        int64
status_parkBrakeIsActive        int64
temperature_ambient           float64
traction_brakePressure        float64
traction_tractionForce        float64
dtype: objec

In [21]:
dataframes['B208_2022-03-25_23-51-19_2022-03-26_03-42-34'].head()

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2022-03-25T23:51:19Z,1648252279,2795.944,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,1,1,293.839,245833.3,0.0
1,2022-03-25T23:51:20Z,1648252280,2717.339,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,1,1,293.461,245833.3,0.0
2,2022-03-25T23:51:21Z,1648252281,2904.655,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,1,1,293.150,245833.3,0.0
3,2022-03-25T23:51:22Z,1648252282,2862.673,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,1,1,293.839,245833.3,0.0
4,2022-03-25T23:51:23Z,1648252283,2927.541,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,1,1,293.461,245833.3,0.0


In [22]:
dataframes['B208_2022-03-25_23-51-19_2022-03-26_03-42-34'].shape

(13876, 26)

In [24]:
dataframes['B208_2022-03-25_23-51-19_2022-03-26_03-42-34'].dtypes

time_iso                       object
time_unix                       int64
electric_powerDemand          float64
gnss_altitude                 float64
gnss_course                   float64
gnss_latitude                 float64
gnss_longitude                float64
itcs_busRoute                  object
itcs_numberOfPassengers       float64
itcs_stopName                  object
odometry_articulationAngle    float64
odometry_steeringAngle        float64
odometry_vehicleSpeed         float64
odometry_wheelSpeed_fl        float64
odometry_wheelSpeed_fr        float64
odometry_wheelSpeed_ml        float64
odometry_wheelSpeed_mr        float64
odometry_wheelSpeed_rl        float64
odometry_wheelSpeed_rr        float64
status_doorIsOpen               int64
status_gridIsAvailable          int64
status_haltBrakeIsActive        int64
status_parkBrakeIsActive        int64
temperature_ambient           float64
traction_brakePressure        float64
traction_tractionForce        float64
dtype: objec

As explained before, `time_iso` should be `datetime` format but instead is `object`. For future analysis it will be better to change the format or to use `time_unix`. The other variables formats are correct for the nature of the variable they represent. I assume this adapts also to the other dataframes. 

In [19]:
diz = defaultdict(int)
# defaultdict(int) initializes the dictionary values to 0

for name in file_names:
    prefix = name[:4]
    diz[prefix] += 1

diz

defaultdict(int, {'B183': 864, 'B208': 545})

* 864 driving missions with bus 183
* 545 driving missions with bus 208

### Nan values check

In [34]:
dataframes['B183_2019-04-30_03-18-56_2019-04-30_08-44-20'].isna().sum()

time_iso                          0
time_unix                         0
electric_powerDemand              0
gnss_altitude                 19328
gnss_course                   19328
gnss_latitude                 19328
gnss_longitude                19328
itcs_busRoute                     0
itcs_numberOfPassengers       19332
itcs_stopName                     0
odometry_articulationAngle        0
odometry_steeringAngle            0
odometry_vehicleSpeed             0
odometry_wheelSpeed_fl            0
odometry_wheelSpeed_fr            0
odometry_wheelSpeed_ml            0
odometry_wheelSpeed_mr            0
odometry_wheelSpeed_rl            0
odometry_wheelSpeed_rr            0
status_doorIsOpen                 0
status_gridIsAvailable            0
status_haltBrakeIsActive          0
status_parkBrakeIsActive          0
temperature_ambient               0
traction_brakePressure            0
traction_tractionForce            0
dtype: int64

In [33]:
dataframes['B208_2022-03-25_23-51-19_2022-03-26_03-42-34'].isna().sum()

time_iso                          0
time_unix                         0
electric_powerDemand              0
gnss_altitude                   115
gnss_course                     110
gnss_latitude                   110
gnss_longitude                  110
itcs_busRoute                     0
itcs_numberOfPassengers       13781
itcs_stopName                     0
odometry_articulationAngle        0
odometry_steeringAngle            0
odometry_vehicleSpeed             0
odometry_wheelSpeed_fl            0
odometry_wheelSpeed_fr            0
odometry_wheelSpeed_ml            0
odometry_wheelSpeed_mr            0
odometry_wheelSpeed_rl            0
odometry_wheelSpeed_rr            0
status_doorIsOpen                 0
status_gridIsAvailable            0
status_haltBrakeIsActive          0
status_parkBrakeIsActive          0
temperature_ambient               0
traction_brakePressure            0
traction_tractionForce            0
dtype: int64

In both datasets, it appears that the columns `gnss_altitude`, `gnss_course`, `gnss_latitude`, and `itcs_numberOfPassengers` have some missing values. The number of missing values varies between the two datasets. It might be worth investigating why these specific columns have missing data and how it could impact your analysis or modeling. 

Based on this two sample I think that there are missing values for the two columns described above. In future analysis it's important to take in consideration this aspect but also to consider that possible other variables can have Null values because for now I only checked two samples.


TODO: 
* Describe a bit the data

# Project

## 1. Extract all trips with `busRoute` 83

As described before the variable `busRoute` should have nan values because from `metadata.head()` I see that some cells have `-`. In order to extract the trips with busRoute 83 I decide to transform substitute the values `-` with a null value because the data for that cell is missing.

In [35]:
metadata.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [39]:
metadata.dtypes

name                             object
busNumber                         int64
startTime_iso                    object
startTime_unix                    int64
endTime_iso                      object
endTime_unix                      int64
drivenDistance                  float64
busRoute                         object
energyConsumption               float64
itcs_numberOfPassengers_mean    float64
itcs_numberOfPassengers_min       int64
itcs_numberOfPassengers_max       int64
status_gridIsAvailable_mean     float64
temperature_ambient_mean        float64
temperature_ambient_min         float64
temperature_ambient_max         float64
dtype: object

In [46]:
metadata['busRoute'] = metadata['busRoute'].replace('-', np.nan)

In [47]:
metadata.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [48]:
metadata.isna().sum()

name                             0
busNumber                        0
startTime_iso                    0
startTime_unix                   0
endTime_iso                      0
endTime_unix                     0
drivenDistance                   0
busRoute                        11
energyConsumption                0
itcs_numberOfPassengers_mean     0
itcs_numberOfPassengers_min      0
itcs_numberOfPassengers_max      0
status_gridIsAvailable_mean      0
temperature_ambient_mean         0
temperature_ambient_min          0
temperature_ambient_max          0
dtype: int64

In [50]:
print(f'busRoute variable contains: {metadata["busRoute"].isna().sum()} missing values')

busRoute variable contains: 11 missing values


In [51]:
metadata.dtypes

name                             object
busNumber                         int64
startTime_iso                    object
startTime_unix                    int64
endTime_iso                      object
endTime_unix                      int64
drivenDistance                  float64
busRoute                         object
energyConsumption               float64
itcs_numberOfPassengers_mean    float64
itcs_numberOfPassengers_min       int64
itcs_numberOfPassengers_max       int64
status_gridIsAvailable_mean     float64
temperature_ambient_mean        float64
temperature_ambient_min         float64
temperature_ambient_max         float64
dtype: object

Let's check the unique values of `busRoute`.

In [52]:
metadata['busRoute'].unique()

array([nan, '31', '33', '72', '46', '32', '83', 'N4', 'N2', 'N1'],
      dtype=object)

The bus routes are not only integers because there are also `N4`, `N2` and `N1`. Probably this trips correspond to routes done in the night. I can keep the data type as a `object` and the format to `string` and I can extract the trips with busRoute 83.

In [78]:
metadata_busRoute_83 = metadata[metadata['busRoute']=='83']
metadata_busRoute_83

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


In [79]:
print(f'The number of trips with bus route 83 is: {metadata_busRoute_83.shape[0]}')

The number of trips with bus route 83 is: 846


In [81]:
# Same of doing 
print(f'The number of trips with bus route 83 is: {len(metadata_busRoute_83)}')

The number of trips with bus route 83 is: 846


## 2. Extract all trips where `busRoute` is not a number 

Not a number mean Nan values or night trip.

In [70]:
# Filter rows where 'busRoute' is not numeric
metadata_busRoute_NotInt = metadata[~metadata['busRoute'].astype(str).str.isnumeric()]
metadata_busRoute_NotInt

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10T03:16:11Z,1557458171,2019-05-10T18:51:37Z,1557514297,210577.00,NaN,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13T03:10:23Z,1557717023,2019-05-13T23:16:13Z,1557789373,267033.80,NaN,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24T02:52:47Z,1558666367,2019-05-24T22:35:11Z,1558737311,263432.60,NaN,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


`df[~df['column_name'].condition]`

`~` is used to negate the condition. In this case I want to extract the rows whether each value in the `busRoute` column is not numeric using the `str.isnumeric()` method.

In [77]:
print(f'The number of trips where BusRoute is not Int is: {metadata_busRoute_NotInt.shape[0]}')

The number of trips where BusRoute is not Int is: 84


In [72]:
# Unique values
metadata_busRoute_NotInt['busRoute'].unique()

array([nan, 'N4', 'N2', 'N1'], dtype=object)

If I don't want to include Nan values

In [73]:
metadata_busRoute_NotInt_NotNan = metadata[~metadata['busRoute'].astype(str).str.isnumeric() & ~metadata['busRoute'].isna()]
metadata_busRoute_NotInt_NotNan

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
533,B183_2021-12-18_23-37-00_2021-12-19_03-38-35,183,2021-12-18T23:37:00Z,1639870620,2021-12-19T03:38:35Z,1639885115,76216.06,N4,481350300.0,9.198582,0,37,0.491653,276.8632,275.15,288.15
553,B183_2022-01-07_23-40-43_2022-01-08_03-31-21,183,2022-01-07T23:40:43Z,1641598843,2022-01-08T03:31:21Z,1641612681,68557.06,N2,453625100.0,4.626984,0,13,0.427488,276.9673,275.15,287.15
554,B183_2022-01-08_23-40-17_2022-01-09_03-35-32,183,2022-01-08T23:40:17Z,1641685217,2022-01-09T03:35:32Z,1641699332,67962.92,N2,475383300.0,7.495495,0,26,0.515514,278.5645,277.15,288.15
561,B183_2022-01-15_23-41-46_2022-01-16_03-40-23,183,2022-01-15T23:41:46Z,1642290106,2022-01-16T03:40:23Z,1642304423,77156.70,N1,525168300.0,6.512500,0,32,0.473809,274.9937,273.15,286.15
568,B183_2022-01-21_23-35-40_2022-01-22_03-26-24,183,2022-01-21T23:35:40Z,1642808140,2022-01-22T03:26:24Z,1642821984,71917.75,N2,455476000.0,5.357143,0,23,0.493608,275.3073,274.15,281.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,434776600.0,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,399773700.0,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,447553400.0,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,451916500.0,20.105260,0,74,0.495739,279.4540,277.15,291.15


In [74]:
print(f'The number of trips where BusRoute is not Int and is not Nan is: {metadata_busRoute_NotInt_NotNan.shape[0]}')

The number of trips where bus route is not Int and is not Nan is: 73


In [75]:
print(f'The number of trips where BusRoute is Int is: {metadata[metadata["busRoute"].astype(str).str.isnumeric()].shape[0]}')

The number of trips where BusRoute is Int is: 1325


## 3. For each (busNumber, busRoute) pair, determine the number of trips

In [84]:
metadata['busNumber'].dtypes

dtype('int64')

`busNumber` is correctly data type integer. 

Let's check the unique values.

In [87]:
metadata['busNumber'].unique()

array([183, 208])

As expected, since that there are 2 driving missions: 183 and 208, `bus Number` has two values: 183 and 208.

To answer to this question I consider a dataframe without Nan values (11 rows deleted). I drop the rows with Nan value on `busRoute` column. As checked before `busNumber` does not have missing values.

In [89]:
metadata_notNan = metadata.dropna()
metadata_notNan

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.11458,4,74,0.855234,287.5443,285.15,293.1500
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.68914,0,55,0.777860,288.7490,280.15,294.1500
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.75357,1,67,0.907342,284.7325,282.15,287.1500
5,B183_2019-05-05_07-41-02_2019-05-05_23-20-07,183,2019-05-05T07:41:02Z,1557042062,2019-05-05T23:20:07Z,1557098407,283206.90,46,1.661700e+09,16.49925,0,74,0.997746,280.1668,277.15,291.1500
6,B183_2019-05-06_03-10-43_2019-05-06_19-20-34,183,2019-05-06T03:10:43Z,1557112243,2019-05-06T19:20:34Z,1557170434,224131.60,31,1.388008e+09,28.03509,0,83,0.871030,282.2435,277.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.80899,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.87940,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.10526,0,74,0.495739,279.4540,277.15,291.1500


In [90]:
metadata_notNan.shape[0]

1398

In [91]:
print(f'The number of rows dropped is: {metadata.shape[0] - metadata_notNan.shape[0]}')

The number of rows dropped is: 11


In [102]:
trips_busNum_busRoute = metadata_notNan.groupby(['busNumber', 'busRoute'], as_index=False).size()

In [103]:
trips_busNum_busRoute = trips_busNum_busRoute.rename(columns={'size': 'numTrips'})

In [104]:
trips_busNum_busRoute

,busNumber,busRoute,numTrips
0,183,31,12
1,183,32,12
2,183,33,130
3,183,46,104
4,183,72,114
5,183,83,441
6,183,N1,10
7,183,N2,19
8,183,N4,11
9,208,31,5


In [106]:
trips_busNum_busRoute['numTrips'].sum()

1398

## 4. For each trip, compute the ratio between the energy consumption and the average number of passengers